In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

### **데이터 로드**

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [4]:
print(train.shape)
print(test.shape)

(256351, 69)
(90067, 68)


### **난자 수**

#### **혼합된 난자 수 설명 가능한지 <- 수집된 신선 난자 수, 해동 난자 수, 난자 출처가 기증 제공**

In [5]:
sum(train['수집된 신선 난자 수'] + train['해동 난자 수'] < train['혼합된 난자 수'])

10014

In [6]:
sum(test['수집된 신선 난자 수'] + test['해동 난자 수'] < test['혼합된 난자 수'])

3558

In [7]:
conditions = train[['수집된 신선 난자 수', '해동 난자 수', '혼합된 난자 수']] > 0
conditions['기증 제공'] = train['난자 출처'] == '기증 제공'

conditions.astype(int).groupby(['혼합된 난자 수', '수집된 신선 난자 수', '해동 난자 수', '기증 제공']).size().reset_index(name='개수')

,혼합된 난자 수,수집된 신선 난자 수,해동 난자 수,기증 제공,개수
0,0,0,0,0,43818
1,0,0,0,1,4954
2,0,0,1,0,13
3,0,0,1,1,6
4,0,1,0,0,4150
5,1,0,0,0,7
6,1,0,0,1,10005
7,1,0,1,0,554
8,1,0,1,1,779
9,1,1,0,0,191947


In [8]:
conditions = test[['수집된 신선 난자 수', '해동 난자 수', '혼합된 난자 수']] > 0
conditions['기증 제공'] = test['난자 출처'] == '기증 제공'

conditions.astype(int).groupby(['혼합된 난자 수', '수집된 신선 난자 수', '해동 난자 수', '기증 제공']).size().reset_index(name='개수')

,혼합된 난자 수,수집된 신선 난자 수,해동 난자 수,기증 제공,개수
0,0,0,0,0,15177
1,0,0,0,1,1667
2,0,0,1,0,5
3,0,0,1,1,3
4,0,1,0,0,1437
5,1,0,0,0,1
6,1,0,0,1,3557
7,1,0,1,0,190
8,1,0,1,1,256
9,1,1,0,0,67725


**정리**
- '기증 받은 난자 수' 정보가 없기 때문에 개수에 대한 정확한 유효성 확인은 불가능
- '난자 출처'가 '기증 제공'인지 여부로 확인하면 설명 가능 (train 7건, test 1건 오류)

**전처리 아이디어**
- 파생변수 - 신선 난자 수만 사용했는지 여부 (100 케이스)

#### **그럼 저장된 신선 난자 수는 뭔데?**

In [9]:
sum(train['수집된 신선 난자 수'] < train['저장된 신선 난자 수'])

1

In [10]:
sum(test['수집된 신선 난자 수'] < test['저장된 신선 난자 수'])

0

In [11]:
conditions = train[['해동 난자 수', '혼합된 난자 수']] > 0
conditions['사용된 신선 난자 수'] = (train['수집된 신선 난자 수'] - train['저장된 신선 난자 수']) > 0
conditions['기증 제공'] = train['난자 출처'] == '기증 제공'

conditions.astype(int).groupby(['혼합된 난자 수', '사용된 신선 난자 수', '해동 난자 수', '기증 제공']).size().reset_index(name='개수')

,혼합된 난자 수,사용된 신선 난자 수,해동 난자 수,기증 제공,개수
0,0,0,0,0,44508
1,0,0,0,1,4954
2,0,0,1,0,13
3,0,0,1,1,6
4,0,1,0,0,3460
5,1,0,0,0,8
6,1,0,0,1,10005
7,1,0,1,0,554
8,1,0,1,1,779
9,1,1,0,0,191946


In [12]:
conditions = test[['해동 난자 수', '혼합된 난자 수']] > 0
conditions['사용된 신선 난자 수'] = (test['수집된 신선 난자 수'] - test['저장된 신선 난자 수']) > 0
conditions['기증 제공'] = test['난자 출처'] == '기증 제공'

conditions.astype(int).groupby(['혼합된 난자 수', '사용된 신선 난자 수', '해동 난자 수', '기증 제공']).size().reset_index(name='개수')

,혼합된 난자 수,사용된 신선 난자 수,해동 난자 수,기증 제공,개수
0,0,0,0,0,15399
1,0,0,0,1,1667
2,0,0,1,0,5
3,0,0,1,1,3
4,0,1,0,0,1215
5,1,0,0,0,1
6,1,0,0,1,3557
7,1,0,1,0,190
8,1,0,1,1,256
9,1,1,0,0,67725


**정리**
- 수집된 신선 난자 중 저장한 난자 개수 맞음 (train 1건, test 0건 오류)
- '수집된 신선 난자 수'에서 '저장된 신선 난자 수'를 제외한 것만 사용했다고 가정해도 '혼합된 난자 수' 설명 가능 (train 8건, test 0건 오류)

**전처리 아이디어**
- 파생변수 - 사용된 신선 난자 수

#### **혼합된 난자 수와 정자 종류 관계**

In [13]:
sum(train['파트너 정자와 혼합된 난자 수'] + train['기증자 정자와 혼합된 난자 수'] != train['혼합된 난자 수'])

7165

In [14]:
sum(train['파트너 정자와 혼합된 난자 수'] + train['기증자 정자와 혼합된 난자 수'] < train['혼합된 난자 수'])

874

In [15]:
sum(test['파트너 정자와 혼합된 난자 수'] + test['기증자 정자와 혼합된 난자 수'] != test['혼합된 난자 수'])

2475

In [16]:
sum(test['파트너 정자와 혼합된 난자 수'] + test['기증자 정자와 혼합된 난자 수'] < test['혼합된 난자 수'])

299

**정리**
- 파트너 정자 + 기증자 정자가 총 '혼합된 난자 수'와 맞지 않음
- 항상 '혼합된 난자 수'가 큰 것으로 보아 출처가 제대로 기록되지 않은 난자들이 있는 것으로 보임

### **배아 수**

#### **신선, 동결, 기증 배아 사용 여부로 설명 가능 - 원래 의도**

In [17]:
conditions = train[['신선 배아 사용 여부', '동결 배아 사용 여부', '기증 배아 사용 여부']] > 0

conditions.astype(int).groupby(['신선 배아 사용 여부', '동결 배아 사용 여부', '기증 배아 사용 여부']).size().reset_index(name='개수')

,신선 배아 사용 여부,동결 배아 사용 여부,기증 배아 사용 여부,개수
0,0,0,0,6294
1,0,1,0,38934
2,0,1,1,987
3,1,0,0,208460
4,1,0,1,1471
5,1,1,0,205


In [18]:
conditions = test[['신선 배아 사용 여부', '동결 배아 사용 여부', '기증 배아 사용 여부']] > 0

conditions.astype(int).groupby(['신선 배아 사용 여부', '동결 배아 사용 여부', '기증 배아 사용 여부']).size().reset_index(name='개수')

,신선 배아 사용 여부,동결 배아 사용 여부,기증 배아 사용 여부,개수
0,0,0,0,2178
1,0,1,0,13487
2,0,1,1,310
3,1,0,0,73526
4,1,0,1,494
5,1,1,0,72


#### **이식된 배아 수 <- 총 생성 배아 수, 해동된 배아 수, 기증 배아 사용 여부로 설명 가능**
- 총 생성 배아 수 - 신선 배아 사용 여부와 연관
- 해동된 배아 수 - 동결 배아 사용 여부와 연관

In [19]:
conditions = train[['이식된 배아 수', '총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부']] > 0

conditions.astype(int).groupby(['이식된 배아 수', '총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부']).size().reset_index(name='개수')

,이식된 배아 수,총 생성 배아 수,해동된 배아 수,기증 배아 사용 여부,개수
0,0,0,0,0,19715
1,0,0,0,1,23
2,0,0,1,0,2816
3,0,0,1,1,36
4,0,1,0,0,19999
5,0,1,0,1,95
6,0,1,1,0,151
7,1,0,0,0,23
8,1,0,0,1,3
9,1,0,1,0,36073


In [20]:
conditions = test[['이식된 배아 수', '총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부']] > 0

conditions.astype(int).groupby(['이식된 배아 수', '총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부']).size().reset_index(name='개수')

,이식된 배아 수,총 생성 배아 수,해동된 배아 수,기증 배아 사용 여부,개수
0,0,0,0,0,6853
1,0,0,0,1,7
2,0,0,1,0,1016
3,0,0,1,1,13
4,0,1,0,0,7007
5,0,1,0,1,31
6,0,1,1,0,54
7,1,0,0,0,11
8,1,0,1,0,12453
9,1,0,1,1,297


#### **총 생성 배아 수는 혼합된 난자 수에서 온 것이고 신선 배아임**

In [21]:
sum(train['혼합된 난자 수'] < train['총 생성 배아 수'])

2

In [22]:
sum(test['혼합된 난자 수'] < test['총 생성 배아 수'])

0

In [23]:
conditions = train[['총 생성 배아 수', '신선 배아 사용 여부']] > 0

conditions.astype(int).groupby(['총 생성 배아 수', '신선 배아 사용 여부']).size().reset_index(name='개수')

,총 생성 배아 수,신선 배아 사용 여부,개수
0,0,0,46161
1,0,1,13479
2,1,0,54
3,1,1,196657


#### **해동된 배아 수는 동결 배아임**

In [24]:
conditions = train[['해동된 배아 수', '동결 배아 사용 여부']] > 0

conditions.astype(int).groupby(['해동된 배아 수', '동결 배아 사용 여부']).size().reset_index(name='개수')

,해동된 배아 수,동결 배아 사용 여부,개수
0,0,0,215966
1,1,0,259
2,1,1,40126


#### **그럼 저장된 배아 수는 뭔데?**

- 추측 : 난자 혼합으로 생성된 배아 (총 생성 배아 수), 해동된 배아, 기증 받은 배아 중 이식하지 않고 저장한 배아 수

In [25]:
sum(train['총 생성 배아 수'] + train['해동된 배아 수'] < train['저장된 배아 수'])

116

In [26]:
sum(train['총 생성 배아 수'] + train['해동된 배아 수'] < train['저장된 배아 수'] + train['이식된 배아 수'])

167

In [27]:
sum(test['총 생성 배아 수'] + test['해동된 배아 수'] < test['저장된 배아 수'])

48

In [28]:
sum(test['총 생성 배아 수'] + test['해동된 배아 수'] < test['저장된 배아 수'] + test['이식된 배아 수'])

67

In [29]:
conditions = train[['저장된 배아 수', '총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부']] > 0

conditions.astype(int).groupby(['저장된 배아 수', '총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부']).size().reset_index(name='개수')

,저장된 배아 수,총 생성 배아 수,해동된 배아 수,기증 배아 사용 여부,개수
0,0,0,0,0,19635
1,0,0,0,1,26
2,0,0,1,0,37622
3,0,0,1,1,974
4,0,1,0,0,113845
5,0,1,0,1,719
6,0,1,1,0,336
7,1,0,0,0,103
8,1,0,1,0,1267
9,1,0,1,1,13


In [30]:
conditions = test[['저장된 배아 수', '총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부']] > 0

conditions.astype(int).groupby(['저장된 배아 수', '총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부']).size().reset_index(name='개수')

,저장된 배아 수,총 생성 배아 수,해동된 배아 수,기증 배아 사용 여부,개수
0,0,0,0,0,6824
1,0,0,0,1,7
2,0,0,1,0,13019
3,0,0,1,1,305
4,0,1,0,0,40371
5,0,1,0,1,237
6,0,1,1,0,119
7,1,0,0,0,40
8,1,0,1,0,450
9,1,0,1,1,5


In [31]:
selected = [
    "수집된 신선 난자 수", "저장된 신선 난자 수", "해동 난자 수",
    "혼합된 난자 수", "총 생성 배아 수", "저장된 배아 수", "해동된 배아 수",
    "이식된 배아 수", "임신 성공 여부", "난자 출처"
]

In [32]:
train.loc[(train['해동된 배아 수'] >0 ) & (train['저장된 배아 수'] > 0), selected]

,수집된 신선 난자 수,저장된 신선 난자 수,해동 난자 수,혼합된 난자 수,총 생성 배아 수,저장된 배아 수,해동된 배아 수,이식된 배아 수,임신 성공 여부,난자 출처
216,0.0,0.0,0.0,0.0,0.0,2.0,9.0,1.0,1,본인 제공
429,5.0,0.0,0.0,5.0,2.0,1.0,1.0,0.0,0,본인 제공
805,0.0,0.0,0.0,0.0,0.0,1.0,9.0,2.0,1,본인 제공
984,0.0,0.0,0.0,0.0,0.0,3.0,5.0,2.0,0,기증 제공
1036,0.0,0.0,0.0,0.0,0.0,1.0,6.0,2.0,1,본인 제공
...,...,...,...,...,...,...,...,...,...,...
255469,6.0,0.0,0.0,5.0,2.0,1.0,1.0,2.0,0,본인 제공
255510,0.0,0.0,0.0,0.0,0.0,1.0,7.0,2.0,0,본인 제공
255869,0.0,0.0,0.0,0.0,0.0,3.0,9.0,2.0,0,본인 제공
255956,0.0,0.0,0.0,0.0,0.0,1.0,7.0,1.0,0,본인 제공


### **혼합된 난자 수 (총 생성 배아 수), 이식된 배아 수가 어떻게 0일 수 있지?**

#### **혼합된 난자 수 (총 생성 배아 수)**

In [33]:
train.loc[train['혼합된 난자 수'] == 0, '임신 성공 여부'].value_counts()

임신 성공 여부
0    37496
1     9154
Name: count, dtype: int64

In [34]:
conditions = train[['혼합된 난자 수', '동결 배아 사용 여부', '기증 배아 사용 여부', '이식된 배아 수']] > 0

conditions.astype(int).groupby(['혼합된 난자 수', '동결 배아 사용 여부', '기증 배아 사용 여부', '이식된 배아 수']).size().reset_index(name='개수')

,혼합된 난자 수,동결 배아 사용 여부,기증 배아 사용 여부,이식된 배아 수,개수
0,0,0,0,0,13111
1,0,0,0,1,16
2,0,0,1,1,3
3,0,1,0,0,2809
4,0,1,0,1,36015
5,0,1,1,0,36
6,0,1,1,1,951
7,1,0,0,0,26761
8,1,0,0,1,174866
9,1,0,1,0,118


In [35]:
conditions = test[['혼합된 난자 수', '동결 배아 사용 여부', '기증 배아 사용 여부', '이식된 배아 수']] > 0

conditions.astype(int).groupby(['혼합된 난자 수', '동결 배아 사용 여부', '기증 배아 사용 여부', '이식된 배아 수']).size().reset_index(name='개수')

,혼합된 난자 수,동결 배아 사용 여부,기증 배아 사용 여부,이식된 배아 수,개수
0,0,0,0,0,4531
1,0,0,0,1,4
2,0,1,0,0,1013
3,0,1,0,1,12431
4,0,1,1,0,13
5,0,1,1,1,297
6,1,0,0,0,9386
7,1,0,0,1,61783
8,1,0,1,0,38
9,1,0,1,1,456


In [36]:
conditions = train[['총 생성 배아 수', '동결 배아 사용 여부', '기증 배아 사용 여부', '이식된 배아 수']] > 0

conditions.astype(int).groupby(['총 생성 배아 수', '동결 배아 사용 여부', '기증 배아 사용 여부', '이식된 배아 수']).size().reset_index(name='개수')

,총 생성 배아 수,동결 배아 사용 여부,기증 배아 사용 여부,이식된 배아 수,개수
0,0,0,0,0,19722
1,0,0,0,1,23
2,0,0,1,0,23
3,0,0,1,1,3
4,0,1,0,0,2809
5,0,1,0,1,36073
6,0,1,1,0,36
7,0,1,1,1,951
8,1,0,0,0,20150
9,1,0,0,1,174859


In [37]:
conditions = test[['총 생성 배아 수', '동결 배아 사용 여부', '기증 배아 사용 여부', '이식된 배아 수']] > 0

conditions.astype(int).groupby(['총 생성 배아 수', '동결 배아 사용 여부', '기증 배아 사용 여부', '이식된 배아 수']).size().reset_index(name='개수')

,총 생성 배아 수,동결 배아 사용 여부,기증 배아 사용 여부,이식된 배아 수,개수
0,0,0,0,0,6856
1,0,0,0,1,11
2,0,0,1,0,7
3,0,1,0,0,1013
4,0,1,0,1,12453
5,0,1,1,0,13
6,0,1,1,1,297
7,1,0,0,0,7061
8,1,0,0,1,61776
9,1,0,1,0,31


#### **이식된 배아 수**

In [38]:
train.loc[train['이식된 배아 수'] == 0, '임신 성공 여부'].value_counts()

임신 성공 여부
0    36514
1       30
Name: count, dtype: int64

### **미세주입**

#### **포함관계 확인**

In [39]:
conditions = train[['이식된 배아 수', '미세주입 배아 이식 수']] > 0

conditions.astype(int).groupby(['이식된 배아 수', '미세주입 배아 이식 수']).size().reset_index(name='개수')

,이식된 배아 수,미세주입 배아 이식 수,개수
0,0,0,42835
1,1,0,102230
2,1,1,111286


In [40]:
conditions = train[['총 생성 배아 수', '미세주입에서 생성된 배아 수']] > 0

conditions.astype(int).groupby(['총 생성 배아 수', '미세주입에서 생성된 배아 수']).size().reset_index(name='개수')

,총 생성 배아 수,미세주입에서 생성된 배아 수,개수
0,0,0,59640
1,1,0,78423
2,1,1,118288


In [41]:
conditions = train[['저장된 배아 수', '미세주입 후 저장된 배아 수']] > 0

conditions.astype(int).groupby(['저장된 배아 수', '미세주입 후 저장된 배아 수']).size().reset_index(name='개수')

,저장된 배아 수,미세주입 후 저장된 배아 수,개수
0,0,0,173157
1,1,0,37428
2,1,1,45766


In [42]:
conditions = train[['혼합된 난자 수', '미세주입된 난자 수']] > 0

conditions.astype(int).groupby(['혼합된 난자 수', '미세주입된 난자 수']).size().reset_index(name='개수')

,혼합된 난자 수,미세주입된 난자 수,개수
0,0,0,52941
1,1,0,81019
2,1,1,122391


#### **대소관계 유효성**

In [43]:
sum(train['미세주입된 난자 수'] < train['미세주입에서 생성된 배아 수'])

2

In [44]:
sum(test['미세주입된 난자 수'] < test['미세주입에서 생성된 배아 수'])

0

In [45]:
sum(train['미세주입에서 생성된 배아 수'] < train['미세주입 후 저장된 배아 수'])

78

In [46]:
sum(train['미세주입에서 생성된 배아 수'] < train['미세주입 배아 이식 수'])

6493

In [47]:
conditions = train[['해동된 배아 수', '기증 배아 사용 여부']] > 0
conditions['미세주입 오류'] = (train['미세주입에서 생성된 배아 수'] < train['미세주입 후 저장된 배아 수']) | (train['미세주입에서 생성된 배아 수'] < train['미세주입 배아 이식 수'])

conditions.astype(int).groupby(['해동된 배아 수', '기증 배아 사용 여부', '미세주입 오류']).size().reset_index(name='개수')

,해동된 배아 수,기증 배아 사용 여부,미세주입 오류,개수
0,0,0,0,214405
1,0,0,1,90
2,0,1,0,1468
3,0,1,1,3
4,1,0,0,32968
5,1,0,1,6430
6,1,1,0,949
7,1,1,1,38


In [48]:
conditions = test[['해동된 배아 수', '기증 배아 사용 여부']] > 0
conditions['미세주입 오류'] = (test['미세주입에서 생성된 배아 수'] < test['미세주입 후 저장된 배아 수']) | (test['미세주입에서 생성된 배아 수'] < test['미세주입 배아 이식 수'])

conditions.astype(int).groupby(['해동된 배아 수', '기증 배아 사용 여부', '미세주입 오류']).size().reset_index(name='개수')

,해동된 배아 수,기증 배아 사용 여부,미세주입 오류,개수
0,0,0,0,75574
1,0,0,1,39
2,0,1,0,494
3,1,0,0,11426
4,1,0,1,2224
5,1,1,0,299
6,1,1,1,11


#### **ICSI와의 관계**

In [49]:
conditions = train[['미세주입된 난자 수', '미세주입에서 생성된 배아 수', '미세주입 후 저장된 배아 수', '미세주입 배아 이식 수']] > 0
conditions['ICSI 여부'] = train['특정 시술 유형'].notna() & train['특정 시술 유형'].str.contains('ICSI')

conditions.astype(int).groupby(['ICSI 여부', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '미세주입 후 저장된 배아 수', '미세주입 배아 이식 수']).size().reset_index(name='개수')

,ICSI 여부,미세주입된 난자 수,미세주입에서 생성된 배아 수,미세주입 후 저장된 배아 수,미세주입 배아 이식 수,개수
0,0,0,0,0,0,126454
1,0,1,0,0,0,104
2,0,1,1,0,0,656
3,0,1,1,1,0,590
4,1,0,0,0,0,1036
5,1,0,0,0,1,6460
6,1,0,0,1,0,2
7,1,0,0,1,1,8
8,1,1,0,0,0,3934
9,1,1,0,0,1,7


In [50]:
train['특정 시술 유형'].str.contains('ICSI')

0          True
1          True
2         False
3          True
4          True
          ...  
256346     True
256347     True
256348     True
256349     True
256350    False
Name: 특정 시술 유형, Length: 256351, dtype: object

### **경과일**

In [51]:
conditions = train[['해동된 배아 수']] > 0
conditions['배아 해동 경과일'] = train['배아 해동 경과일'].notna()

conditions.astype(int).groupby(['해동된 배아 수', '배아 해동 경과일']).size().reset_index(name='개수')

,해동된 배아 수,배아 해동 경과일,개수
0,0,0,215963
1,0,1,3
2,1,0,19
3,1,1,40366


In [52]:
conditions = test[['해동된 배아 수']] > 0
conditions['배아 해동 경과일'] = test['배아 해동 경과일'].notna()

conditions.astype(int).groupby(['해동된 배아 수', '배아 해동 경과일']).size().reset_index(name='개수')

,해동된 배아 수,배아 해동 경과일,개수
0,0,0,76105
1,0,1,2
2,1,0,12
3,1,1,13948


In [53]:
conditions = train[['해동 난자 수']] > 0
conditions['난자 해동 경과일'] = train['난자 해동 경과일'].notna()

conditions.astype(int).groupby(['해동 난자 수', '난자 해동 경과일']).size().reset_index(name='개수')

,해동 난자 수,난자 해동 경과일,개수
0,0,0,254904
1,0,1,2
2,1,0,11
3,1,1,1434


In [54]:
conditions = test[['해동 난자 수']] > 0
conditions['난자 해동 경과일'] = test['난자 해동 경과일'].notna()

conditions.astype(int).groupby(['해동 난자 수', '난자 해동 경과일']).size().reset_index(name='개수')

,해동 난자 수,난자 해동 경과일,개수
0,0,0,89573
1,1,0,2
2,1,1,492


#### **4-3. 난자 배아 수 변수들과 경과일 변수들 간의 유효성 (결측치 퐁당퐁당에서 시작)**

In [5]:
ivf_train = train[train['시술 유형'] == 'IVF']
ivf_test = test[test['시술 유형'] == 'IVF']

In [6]:
ivf_train.loc[:, ['난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 해동 경과일', '배아 이식 경과일']] = \
ivf_train.loc[:, ['난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 해동 경과일', '배아 이식 경과일']].fillna(-1)


1) [확인해야할 동결 배아의 2가지 case - 난자 배아 수]

In [7]:
conditions = ivf_train[['총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부', '이식된 배아 수']] > 0

conditions.astype(int).groupby(['총 생성 배아 수', '해동된 배아 수', '기증 배아 사용 여부', '이식된 배아 수']).size().reset_index(name='개수')

,총 생성 배아 수,해동된 배아 수,기증 배아 사용 여부,이식된 배아 수,개수
0,0,0,0,0,13424
1,0,0,0,1,23
2,0,0,1,0,23
3,0,0,1,1,3
4,0,1,0,0,2816
5,0,1,0,1,36073
6,0,1,1,0,36
7,0,1,1,1,951
8,1,0,0,0,19999
9,1,0,0,1,174758


- case 1 : 신선 난자 x, 동결 배아 o - `해동된 배아 수`  : 1 , `총 생성 배아 수`  : 0 ⇒ 39876건 (2816 + 36073 + 36 + 951 = 39876) 

- case 2 : 신선 난자 o , 동결 배아 o  - `해동된 배아 수`  : 1, `총 생성 배아 수`  : 1 ⇒509건 (151 + 358 = 509)

2) [동결 배아의 2가지 case 확인 - 경과일]

In [8]:
conditions = ivf_train[['난자 채취 경과일', '난자 혼합 경과일', '배아 해동 경과일']] > -1

conditions.astype(int).groupby(['난자 채취 경과일', '난자 혼합 경과일', '배아 해동 경과일']).size().reset_index(name='개수')

,난자 채취 경과일,난자 혼합 경과일,배아 해동 경과일,개수
0,0,0,0,29
1,0,0,1,39791
2,0,1,0,11366
3,0,1,1,11
4,1,0,0,7607
5,1,0,1,17
6,1,1,0,190689
7,1,1,1,550


- case 1 : `배아 해동 경과일`  : 1 , `난자 혼합 경과일`  : 0,  `난자 채취 경과일`  : 0 ⇒ 39791건 # 85건 오류
- case 2 : `배아 해동 경과일`  : 1 , `난자 혼합 경과일`  : 1,  `난자 채취 경과일`  : 1⇒ 550건  # 41건 오류

In [9]:
(ivf_train['난자 출처'] == '기증 제공').sum()

15769

- 2	0	1	0	11366 인덱스 2번 케이스에 대한 설명 
- 난자 채취를 안 했는데 혼합한 케이스? (난자 채취 x, 혼합 o , 해동 x) 11366건 에 대한 설명
⇒  `난자 출처 == '기증 제공'`  15769건  에 해당하지 않을까

#### **경과일 전체보기** 

In [11]:
df_sorted = conditions.astype(int).groupby(
    ['난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 해동 경과일', '배아 이식 경과일']
).size().reset_index(name='개수').sort_values(
    by=['배아 이식 경과일', '배아 해동 경과일', '난자 혼합 경과일', '난자 해동 경과일', '난자 채취 경과일'],
    ascending=[True, True, True, True, True]
).reset_index(drop=True)

df_sorted


,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 해동 경과일,배아 이식 경과일,개수
0,1,0,0,0,0,7238
1,0,1,0,0,0,10
2,0,0,1,0,0,1153
3,1,0,1,0,0,25275
4,0,1,1,0,0,211
5,1,1,1,0,0,24
6,0,0,0,1,0,3215
7,1,0,0,1,0,4
8,1,0,1,1,0,141
9,0,1,1,1,0,2


##### **배아 이식 경과일과 이식된 배아 수 유효성 확인**

In [ ]:
# 1. 배아 이식 경과일 - train
(ivf_train['배아 이식 경과일'] >= 0).sum()

212785

In [ ]:
# 1. 이식된 배아 수 -train
(ivf_train['이식된 배아 수'] > 0).sum()

213516

In [ ]:
# 1. 배아 이식 경과일 -test
(ivf_test['배아 이식 경과일'] >= 0).sum()

74821

In [ ]:
# 1. 이식된 배아 수 -test
(ivf_test['이식된 배아 수'] > 0).sum()

75086